# This script is estimating the domain size of active material from cross-sectional SEM

## How should the format be for the files?

Just the .tif files from the JEOL SEM will work fine.

## How do I use this script? I don't know any coding...

No problem, this one is a bit more advanced than the other scripts but not much more

### Step 1

You should see a blue bar to the left of this text, this refers to which part of the code the system will read next. To have the system read the code, just hit Shift+Enter and it will go to the next "Cell"

### Step 2

Each cell will have a Blurb at the top with a # before it, this is meant to tell you what the cell is doing. There are some cells that you don't need to worry about changing and others that only need very minor input.

Read the blurb, if it says "#Don't worry about it", then just hit Shift+Enter

You can tell if the cell is done running by either an output such as a number or plot OR it will read "DONE"

In [ ]:
#Author: Wess van den Bergh
#Date Modified: May 16, 2024
#Environment: v2_Pharmakinetics
#Verified By: 

#Notes, known errors, desired features

# May still spit out WARNINGS
# Not tested with other images so robustness with range of image qualities is unknown
# Not directly compared with other methods -- some imageJ comparisons that are roughly inline

print("DONE")

In [ ]:
#Don't worry about it, just hit Shift+Enter

#importing necessary functions
import skimage as ski
import matplotlib.pyplot as plt
import numpy as np
from tkinter import*
import pyclesperanto_prototype as cle
import pandas as pd
import stackview
import os
import warnings
import cmcrameri as cmc

#Specific functions
from napari_segment_blobs_and_things_with_membranes import voronoi_otsu_labeling
from scipy.ndimage import binary_fill_holes
from napari_skimage_regionprops import regionprops_table
from napari_simpleitk_image_processing import label_statistics
from scipy.stats import gaussian_kde

def import_then_process_img():
    """ grabs the image with a GUI for domain size analysis """
    # Create Tk root
    root = Tk()
    # Hide the main window
    root.withdraw()
    root.call('wm', 'attributes', '.', '-topmost', True)
    
    from tkinter import filedialog
    img_filepath = filedialog.askopenfilename(multiple=False)
    
    %gui tk
    
    if '.tif' not in repr(img_filepath) and '.tiff' not in repr(img_filepath):
        wanrings.warn("image is not a .tiff or .tif file")
    else:
        pass
    
    #run a test to see if the file is RGBA or RGB AND turn the image to grayscale
    try:
        temp_grayscale_image = ski.color.rgb2gray(ski.color.rgba2rgb(ski.io.imread(img_filepath)))
        try:
            temp_grayscale_image = ski.color.rgb2gray(ski.io.imread(img_filepath))
        except:
            pass
    except:
        warnings.warn("not RGB or RGBA image")
    
    plt.imshow(temp_grayscale_image)
    plt.show()
    
    scalebar_point = int(input("What is an intersecting X-value with the scalebar? "))
    scalebar_size = int(input("What is the size of the scalebar? "))
    
    plt.clf()
    
    img_list = img_remove_watermark_set_scale(temp_grayscale_image, scalebar_point, scalebar_size) #returns the image and the microns per pixel scale of the image as a float/scalar

    crop_yn = input("Does the image need to be cropped? (y/n) ")

    if crop_yn.lower() in ['true', '1', 't', 'y', 'yes']:
        crop_lower_input_value = int(input("What is the lower-end crop Y (bottom portion) value? "))
        crop_upper_input_value = int(input("What is the upper-end crop Y (top portion) value? "))
        temp_image_grayscale = img_crop(img_list[0], crop_upper = crop_upper_input_value, crop_lower = crop_lower_input_value)

    else:
        temp_image_grayscale = img_list[0]
    
    return temp_image_grayscale, img_list[1]

def img_remove_watermark_set_scale(temp_grayscale_image, scalebar_point, scalebar_size):
    """ removes the watermark from the SEM image and converts it to grayscale """

    #finding scale scalebar size in pixels
    j = temp_grayscale_image.shape[0] - 1
    while temp_grayscale_image[j, scalebar_point] != 1.0:
        j -= 1
    j -= 3 #not sure why I wrote this, either as precaution or to account for something with the scalebar
    
    while temp_grayscale_image[j, scalebar_point] == 1.0:
        scalebar_point -= 1
    scalebar_point_start = scalebar_point
    
    scalebar_point += 1
    while temp_grayscale_image[j, scalebar_point] == 1.0:
        scalebar_point += 1
    scalebar_point_end = scalebar_point
    
    scalebar_pixel_length = scalebar_point_end - scalebar_point_start
    micron_per_pixel = scalebar_size / scalebar_pixel_length

    #crop image to remove watermarks
    i = temp_grayscale_image.shape[0] - 1
    while temp_grayscale_image[i, 3] == 0.0:
        i -= 1
    temp_grayscale_image = temp_grayscale_image[0:i, :]
    
    plt.minorticks_on()
    plt.tick_params(axis='both', which='minor', length=4, width=1, direction='out')
    
    plt.imshow(temp_grayscale_image)
    plt.show()

    return [temp_grayscale_image, micron_per_pixel]

def img_crop(temp_grayscale_image, crop_upper = 0, crop_lower = 0):
    """ has the user input the y-values to crop the image to remove anything that is not of interest """
    
    if crop_upper != 0:
        temp_grayscale_image = temp_grayscale_image[crop_upper:-1, :]
        
    if crop_lower != 0 and crop_upper != 0:
        temp_grayscale_image = temp_grayscale_image[0:(crop_lower-crop_upper), :]
    elif crop_lower != 0 and crop_upper == 0:
        temp_grayscale_image = temp_grayscale_image[0:(crop_lower), :]

    plt.minorticks_on()
    plt.tick_params(axis='both', which='minor', length=4, width=1, direction='out')
    plt.imshow(temp_grayscale_image)
    plt.show()
    return temp_grayscale_image



def try_all_thresholds(temp_grayscale_image):
    """ spits out an array of threshold methods that the user might find interesting """
    fig, ax = ski.filters.try_all_threshold(temp_grayscale_image, figsize = (20, 16), verbose = False)
    plt.show()

    threshold_selection = input("By name or threshold number, what is the best option for you? ")

    try:
        float(threshold_selection)
    except ValueError:
        pass

    temp_image_binary = img_threshold(temp_grayscale_image, threshold_selection)
    
    return temp_image_binary

def img_threshold(temp_grayscale_image, selctn):
    """ applies the threshold of choice or takes a float value for thresholding if none are satisfactory """
    if selctn.lower() == "isodata":
        thresh = ski.filters.threshold_isodata(temp_grayscale_image)
    elif selctn.lower() == "li":
        thresh = ski.filters.threshold_li(temp_grayscale_image)
    elif selctn.lower() == "mean":
        thresh = ski.filters.threshold_mean(temp_grayscale_image)
    elif selctn.lower() == "minimum":
        thresh = ski.filters.threshold_minimum(temp_grayscale_image)
    elif selctn.lower() == "otsu":
        thresh = ski.filters.threshold_otsu(temp_grayscale_image)
    elif selctn.lower() == "triangle":
        thresh = ski.filters.threshold_triangle(temp_grayscale_image)
    elif selctn.lower() == "yen":
        thresh = ski.filters.threshold_yen(temp_grayscale_image)
    elif isinstance(selctn, float):
        thresh = selctn

    temp_image_binary = temp_grayscale_image > thresh
    
    return temp_image_binary


def img_erosion_n_fill(temp_image_binary, erosion_range = 4):
    """ 
    shows an array of erosion value options to see which erosion value is best to remove excess
    """
    
    fig, axs = plt.subplots(erosion_range, 1, figsize=(15,20))
    cle.imshow(temp_image_binary, plot=axs[0])
    axs[0].set_title('Binary image')
    for i in range(1, erosion_range):
        temp_eroded_binary = ski.morphology.binary_erosion(temp_image_binary, ski.morphology.disk(i))
        stackview.imshow(temp_eroded_binary, plot=axs[i])
        axs[i].set_title('Eroded r={}'.format(i))
    plt.tight_layout()
    plt.show()

    erosion_value = int(input("What erosion value do you want to apply? (0 for no erosion): "))
    eroded_binary = img_erosion_apply(temp_image_binary, erosion_value)
    eroded_fill_treated_binary = img_hole_fill(eroded_binary)
    second_erosion_decision = input("After hole filling, does the image need a second round of erosion? (y/n): ")
    
    plt.clf

    if second_erosion_decision.lower() in ['true', '1', 't', 'y', 'yes']:
        
        fig, axs = plt.subplots(erosion_range, 1, figsize=(15,20))
        cle.imshow(eroded_fill_treated_binary, plot=axs[0])
        axs[0].set_title('Binary image')
        for i in range(1, erosion_range):
            temp_eroded_binary = ski.morphology.binary_erosion(eroded_fill_treated_binary, ski.morphology.disk(i))
            stackview.imshow(eroded_fill_treated_binary, plot=axs[i])
            axs[i].set_title('Eroded r={}'.format(i))
        plt.tight_layout()
        plt.show()

        second_erosion_value = int(input("What erosion value do you want to apply? (0 for no erosion): "))
        second_eroded_fill_treated_binary = img_erosion_apply(eroded_fill_treated_binary, second_erosion_value)
        return second_eroded_fill_treated_binary
    else:
        return eroded_fill_treated_binary

def img_erosion_apply(temp_image_binary, erosion_value):
    """ 
    used to wash out extra stuff/noise picked up that is not particles
    note that it also erodes edges of particles so use sparingly
    """
    eroded_binary = ski.morphology.binary_erosion(temp_image_binary, ski.morphology.disk(erosion_value))
    return eroded_binary



def img_hole_fill(temp_image_binary):
    """ 
    fills the holes in particles, this may not be necessary or undesired so consider it optional
    """
    
    fill_decision = input("Do you want to fill in the holes of the particles?: (y/n)")
    
    if fill_decision.lower() in ['true', '1', 't', 'y', 'yes']:
    
        filled_binary = binary_fill_holes(temp_image_binary)

        fig, axs = plt.subplots(2, 1, figsize=(15,15))
        cle.imshow(temp_image_binary, plot=axs[0])
        axs[0].set_title('Binary image')

        cle.imshow(filled_binary, plot=axs[1])
        axs[1].set_title('Holes filled')

        return filled_binary
    
    else:
        return temp_image_binary


def img_binary_labeling(temp_image_binary, 
                        spot_sigma_val = 15,
                        outline_sigma_val = 0):
    """ 
    IDs individual particles with two different methods as neither method is perfect, thus two imperfect ones are used
    """
    voroni_otsu_lbls = voronoi_otsu_labeling(temp_image_binary, 
                                             spot_sigma = spot_sigma_val, 
                                             outline_sigma = outline_sigma_val)
    cle.imshow(voroni_otsu_lbls, labels = True)
    von_neumann_lbls = ski.measure.label(temp_image_binary, connectivity = 1)
    cle.imshow(von_neumann_lbls, labels=True)

    return [von_neumann_lbls, voroni_otsu_lbls]


def img_binary_enframing(temp_grayscale_image, von_neumann_lbls, voroni_otsu_lbls, micron_per_pixel):
    """ 
    Applies data values to the particles idenified -- uses two enframing methods to grab the necessary values
    """
    voroni_otsu_df_perim = pd.DataFrame(label_statistics(temp_grayscale_image, voroni_otsu_lbls,  
                                  shape=False, 
                                  perimeter=True, 
                                  position=False,
                                  moments=False))
    
    von_neumann_df_perim = pd.DataFrame(label_statistics(temp_grayscale_image, von_neumann_lbls,  
                                  shape=False, 
                                  perimeter=True, 
                                  position=False,
                                  moments=False))
    
    voroni_otsu_df = pd.DataFrame(regionprops_table(temp_grayscale_image , voroni_otsu_lbls, 
                                               perimeter = True, 
                                               shape = True, 
                                               position=True,
                                               moments=False))
    
    von_neumann_df = pd.DataFrame(regionprops_table(temp_grayscale_image , von_neumann_lbls, 
                                               perimeter = True, 
                                               shape = True, 
                                               position=True,
                                               moments=False))
    
    voroni_otsu_df = pd.concat([voroni_otsu_df, voroni_otsu_df_perim['perimeter_on_border_ratio']], axis = 1)
    von_neumann_df = pd.concat([von_neumann_df, von_neumann_df_perim['perimeter_on_border_ratio']], axis = 1)
    
    voroni_otsu_df['equivalent_diameter'] = voroni_otsu_df['equivalent_diameter'] * micron_per_pixel
    von_neumann_df['equivalent_diameter'] = von_neumann_df['equivalent_diameter'] * micron_per_pixel
    voroni_otsu_df['area'] = voroni_otsu_df['area'] * (micron_per_pixel ** 2)
    von_neumann_df['area'] = von_neumann_df['area'] * (micron_per_pixel ** 2)

    return [von_neumann_df, voroni_otsu_df]



def cutoff_particle_size(von_neumann_df, voroni_otsu_df):
    """ 
    Noise will be ID'd as particles as well, this is meant to set a cutoff to remove that noise to get a more accurate analysis
    """
    fig, ax = plt.subplots()

    plt.ecdf(voroni_otsu_df['equivalent_diameter'], label = 'Voroni Otsu')
    plt.ecdf(von_neumann_df['equivalent_diameter'], label = 'Von Neumann')
    
    ax.set_xlim([0, 5])
    
    plt.xlabel('Equiv. Diameter (microns)')
    plt.ylabel('ECDF')
    
    plt.legend()
    
    plt.show()
    
    print("Average value of Von Neumann labeling: {}".format(von_neumann_df['equivalent_diameter'].mean()))
    print("Average value of Voroni Otsu labeling: {}".format(voroni_otsu_df['equivalent_diameter'].mean()))
    

def cutoff_particles_on_edge(von_neumann_filtered_df, voroni_otsu_filtered_df):
    """ 
    Particles at the edge will be ID'd as particles as well, this is meant to set a cutoff to remove that noise to get a more accurate analysis
    """
    fig, ax = plt.subplots()
    
    plt.ecdf(voroni_otsu_filtered_df['perimeter_on_border_ratio'], label = 'Voroni Otsu')
    plt.ecdf(von_neumann_filtered_df['perimeter_on_border_ratio'], label = 'Von Neumann')
    
    plt.xlabel('Ratio of Perimeter on Border')
    plt.ylabel('ECDF')
    plt.legend()
    
    plt.show()
    
    print("Average value of Voroni Otsu labeling: {}".format(voroni_otsu_filtered_df['perimeter_on_border_ratio'].mean()))
    print("Average value of Von Neumann labeling: {}".format(von_neumann_filtered_df['perimeter_on_border_ratio'].mean()))


def cutoff_particle_apply(von_neumann_df, voroni_otsu_df, cutoff_type, cutoff_value):
    """ 
    Application of cutoff value for particle size
    """
    if cutoff_type == "size":
        von_neumann_df = von_neumann_df[von_neumann_df['equivalent_diameter'] > cutoff_value]
        voroni_otsu_df = voroni_otsu_df[voroni_otsu_df['equivalent_diameter'] > cutoff_value]
        
    elif cutoff_type == "edge":
        von_neumann_df = von_neumann_df[von_neumann_df['perimeter_on_border_ratio'] < cutoff_value]
        voroni_otsu_df = voroni_otsu_df[voroni_otsu_df['perimeter_on_border_ratio'] < cutoff_value]

    return [von_neumann_df, voroni_otsu_df]   

def finalize_data(von_neumann_filtered2_df, voroni_otsu_filtered2_df):
    """ 
    seeks to determine if there are any remaining outliers albiet the cutoff for which is pretty generous
    """

    # After removing speck outliers then apply MAD outlier filter with cutoff value = 3 (http://dx.doi.org/10.1016/j.jesp.2013.03.013)
    von_neumann_filtered2_df['preMAD_diam'] = abs(von_neumann_filtered2_df['equivalent_diameter'] - von_neumann_filtered2_df['equivalent_diameter'].median()) 
    voroni_otsu_filtered2_df['preMAD_diam'] = abs(voroni_otsu_filtered2_df['equivalent_diameter'] - voroni_otsu_filtered2_df['equivalent_diameter'].median())
    
    von_neumann_filtered2_df['MAD_diam'] = (von_neumann_filtered2_df['equivalent_diameter'] - von_neumann_filtered2_df['equivalent_diameter'].median()) / (1.4826 * von_neumann_filtered2_df['preMAD_diam'].median())
    voroni_otsu_filtered2_df['MAD_diam'] = (voroni_otsu_filtered2_df['equivalent_diameter'] - voroni_otsu_filtered2_df['equivalent_diameter'].median()) / (1.4826 * voroni_otsu_filtered2_df['preMAD_diam'].median())
    
    von_neumann_filtered3_df = von_neumann_filtered2_df[abs(von_neumann_filtered2_df['perimeter_on_border_ratio']) < 3]
    voroni_otsu_filtered3_df = voroni_otsu_filtered2_df[abs(voroni_otsu_filtered2_df['perimeter_on_border_ratio']) < 3]
    
    print("Number of outliers removed from Von Neumman method: {}".format(len(von_neumann_filtered2_df.index) % len(von_neumann_filtered3_df.index)))
    print("Number of outliers removed from Voroni Otsu method: {}".format(len(voroni_otsu_filtered2_df.index) % len(voroni_otsu_filtered3_df.index)))

    return [von_neumann_filtered3_df, voroni_otsu_filtered3_df]



def plot_and_process_particle_data(temp_grayscale_image, 
                                   von_neumann_filtered3_df, 
                                   von_neumann_lbls, 
                                   voroni_otsu_filtered3_df, 
                                   voroni_otsu_lbls,
                                   experiment_id_for_file = None,
                                   consolidated_data = False
                                  ):

    if consolidated_data == False:
        plot_data(temp_grayscale_image, 
                  von_neumann_filtered3_df, 
                  von_neumann_lbls, 
                  voroni_otsu_filtered3_df, 
                  voroni_otsu_lbls,
                  consolidated_plot_data = False
                 )
    
        if isinstance(experiment_id_for_file, str) == True:
            save_domain_size_df(von_neumann_filtered3_df,
                                df_type = "von_neumann",
                                experiment_id = experiment_id_for_file)
        
            save_domain_size_df(voroni_otsu_filtered3_df,
                                df_type = "voroni_otsu",
                                experiment_id = experiment_id_for_file)

    if consolidated_data == True:
        plot_data(temp_grayscale_image = None, 
                  von_neumann_filtered3_df = von_neumann_filtered3_df, 
                  von_neumann_lbls = None, 
                  voroni_otsu_filtered3_df = voroni_otsu_filtered3_df, 
                  voroni_otsu_lbls = None,
                  consolidated_plot_data = True
                 )

def plot_data(temp_grayscale_image, 
              von_neumann_filtered3_df, 
              von_neumann_lbls, 
              voroni_otsu_filtered3_df, 
              voroni_otsu_lbls,
              ax_font_size = 14,
              consolidated_plot_data = False
             ):
    """ 
    Plots the data and reports the statistics 
    """

    BINS_FOR_HISTOGRAM = np.linspace(0.0, 10, 41, endpoint=True)

    if temp_grayscale_image.all(None) == False: #If there is an image rather than a None input
        # Hit shift+enter, here are the domains contained by "bounding boxes" as well as the stats and violin plots
        # Bounding Box plot for von neumann
        plt.imshow(temp_grayscale_image, cmap = 'gray')
        for index, row in von_neumann_filtered3_df.iterrows():
            bbox_min = (row['bbox-1'], row['bbox-0'])
            bbox_max = (row['bbox-3'], row['bbox-2'])
            width = bbox_max[0] - bbox_min[0]
            height = bbox_max[1] - bbox_min[1]
            rect = plt.Rectangle(bbox_min, width, height, edgecolor='r', facecolor='none')
            plt.gca().add_patch(rect)
        plt.title('Von Neumann Domains')
        plt.show()
        
        # Bounding Box plot for von neumann
        plt.imshow(cle.pull(von_neumann_lbls), cmap='nipy_spectral')  # Convert labeled_image to numpy array before plotting
        for index, row in von_neumann_filtered3_df.iterrows():
            bbox_min = (row['bbox-1'], row['bbox-0'])
            bbox_max = (row['bbox-3'], row['bbox-2'])
            width = bbox_max[0] - bbox_min[0]
            height = bbox_max[1] - bbox_min[1]
            rect = plt.Rectangle(bbox_min, width, height, edgecolor='w', facecolor='none')
            plt.gca().add_patch(rect)
        plt.title('Von Neumann Domains_2')
        plt.show()
        
        # Bounding Box plot for von neumann
        plt.imshow(temp_grayscale_image, cmap = 'gray')
        for index, row in voroni_otsu_filtered3_df.iterrows():
            bbox_min = (row['bbox-1'], row['bbox-0'])
            bbox_max = (row['bbox-3'], row['bbox-2'])
            width = bbox_max[0] - bbox_min[0]
            height = bbox_max[1] - bbox_min[1]
            rect = plt.Rectangle(bbox_min, width, height, edgecolor='b', facecolor='none')
            plt.gca().add_patch(rect)   
        plt.title('Voroni Otsu Domains')
        plt.show()
        
        # Bounding Box plot for von neumann
        plt.imshow(cle.pull(voroni_otsu_lbls), cmap='nipy_spectral')  # Convert labeled_image to numpy array before plotting
        for index, row in voroni_otsu_filtered3_df.iterrows():
            bbox_min = (row['bbox-1'], row['bbox-0'])
            bbox_max = (row['bbox-3'], row['bbox-2'])
            width = bbox_max[0] - bbox_min[0]
            height = bbox_max[1] - bbox_min[1]
            rect = plt.Rectangle(bbox_min, width, height, edgecolor='w', facecolor='none')
            plt.gca().add_patch(rect)
        plt.title('Voroni Otsu Domains_2')
        plt.show()


    if consolidated_plot_data == True:
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        root.call('wm', 'attributes', '.', '-topmost', True)
        
        from tkinter import filedialog
        df_csvs = filedialog.askopenfilename(multiple=True)
        
        %gui tk
        
        von_neumann_dfs = []
        voroni_otsu_dfs = []
        
        for i, path in enumerate(df_csvs): 
            if "von_neumann" in os.path.basename(path):
                von_neumann_dfs.append(pd.read_csv(path))
            elif "voroni_otsu" in os.path.basename(path):
                voroni_otsu_dfs.append(pd.read_csv(path))
        
        von_neumann_filtered3_df = pd.concat(von_neumann_dfs, ignore_index = True, sort = False)
        voroni_otsu_filtered3_df = pd.concat(voroni_otsu_dfs, ignore_index = True, sort = False)


    # Number-based stats on each measurement method
    print("Von Neumann: Equivalenet Diameter Stats, number-based")
    von_neumann_stats = von_neumann_filtered3_df['equivalent_diameter'].describe(percentiles = [0.10, 0.50, 0.90])
    von_neumann_stats = pd.concat([von_neumann_stats, 
                                   pd.Series([(von_neumann_stats["90%"] - von_neumann_stats["10%"]) / von_neumann_stats["10%"]], 
                                             index = ['span'])
                                  ]
                                 )
    print(von_neumann_stats)
    print(" ")

    
    print("Voroni Otsu: Equivalenet Diameter Stats, number-based")
    voroni_otsu_stats = voroni_otsu_filtered3_df['equivalent_diameter'].describe(percentiles = [0.10, 0.50, 0.90])
    voroni_otsu_stats = pd.concat([voroni_otsu_stats, 
                                   pd.Series([(voroni_otsu_stats["90%"] - voroni_otsu_stats["10%"]) / voroni_otsu_stats["10%"]], 
                                             index = ['span'])
                                  ]
                                 )
    print(voroni_otsu_stats)
    print(" ")
    
    #Histogram PSD plots for NUMBER VALUES
    fig_num_psd, ax_num_psd = plt.subplots()
    voroni_otsu_filtered3_df['equivalent_diameter'].plot(kind = 'density', 
                                                         color = cmc.cm.vikO(0.15),
                                                         lw = 3
                                                        )
    von_neumann_filtered3_df['equivalent_diameter'].plot(kind = 'density', 
                                                         color = cmc.cm.vikO(0.80),
                                                         lw = 3
                                                        )
    
    plt.hist(voroni_otsu_filtered3_df['equivalent_diameter'], 
             bins = BINS_FOR_HISTOGRAM, 
             density = True, 
             alpha = 0.3, 
             color = cmc.cm.vikO(0.15)
            )
    plt.hist(von_neumann_filtered3_df['equivalent_diameter'], 
             bins = BINS_FOR_HISTOGRAM, 
             density = True, 
             alpha = 0.3,
             color = cmc.cm.vikO(0.80)
            )
    
    plt.xlabel('#-wt\'d Equivalent Diameter (microns)', fontsize = ax_font_size, fontweight = 'bold')
    plt.ylabel('Density',  fontsize = ax_font_size, fontweight = 'bold')
    plt.xticks(fontsize=ax_font_size, fontweight='bold')
    plt.yticks(fontsize=ax_font_size, fontweight='bold')
    plt.box(True)
    plt.gca().spines['top'].set_linewidth(4)    # Set top spine thickness
    plt.gca().spines['bottom'].set_linewidth(4) # Set bottom spine thickness
    plt.gca().spines['left'].set_linewidth(4)   # Set left spine thickness
    plt.gca().spines['right'].set_linewidth(4)  # Set right spine thickness
    plt.tick_params(axis='both', direction='in', length=10)  # Set the length of tick marks
    
    ax_num_psd.set_ylim([0, 1])
    ax_num_psd.set_xlim([0, 10])
    
    num_legend = plt.legend(labels = ['voroni_otsu', 'von neumann'], framealpha=1, frameon=True, fontsize=ax_font_size - 4)
    frame = num_legend.get_frame()
    frame.set_edgecolor('black')  # Set legend box color
    frame.set_linewidth(2)        # Set legend box thickness
    for text in num_legend.get_texts():
        text.set_fontweight('bold')
    plt.show
    
    #Histogram + PSD plots for volume-based analysis
    fig_vol_psd, ax_vol_psd = plt.subplots()

    kde_voroni_otsu = gaussian_kde(voroni_otsu_filtered3_df['equivalent_diameter'], 
                                   bw_method='scott', 
                                   weights = ((4/3) * np.pi) * voroni_otsu_filtered3_df['equivalent_diameter']**3)  # Use Scott's bandwidth method for automatic adjustment
    
    x_values_voroni_otsu = np.linspace(min(voroni_otsu_filtered3_df['equivalent_diameter']), max(voroni_otsu_filtered3_df['equivalent_diameter']), 100)
    y_values_voroni_otsu = kde_voroni_otsu(x_values_voroni_otsu)
    y_values_voroni_otsu /= np.trapezoid(y_values_voroni_otsu, x_values_voroni_otsu)  # Normalizing the area under the curve to 1
    
    
    kde_von_neumann = gaussian_kde(von_neumann_filtered3_df['equivalent_diameter'], 
                                   bw_method='scott', 
                                   weights = ((4/3) * np.pi) * von_neumann_filtered3_df['equivalent_diameter']**3)  # Use Scott's bandwidth method for automatic adjustment
    
    x_values_von_neumann = np.linspace(min(von_neumann_filtered3_df['equivalent_diameter']), max(von_neumann_filtered3_df['equivalent_diameter']), 100)
    y_values_von_neumann = kde_von_neumann(x_values_von_neumann)
    y_values_von_neumann /= np.trapezoid(y_values_von_neumann, x_values_von_neumann)  # Normalizing the area under the curve to 1
    
    # Step 4: Plot the KDE
    plt.plot(x_values_voroni_otsu, 
             y_values_voroni_otsu, 
             color = cmc.cm.romaO(0.80),
             lw = 3
            )
    plt.plot(x_values_von_neumann, 
             y_values_von_neumann, 
             color = cmc.cm.romaO(0.15),
             lw = 3
            )
    
    plt.hist(voroni_otsu_filtered3_df['equivalent_diameter'], 
             bins = BINS_FOR_HISTOGRAM,
             density = True, 
             weights = ((4/3) * np.pi) * voroni_otsu_filtered3_df['equivalent_diameter']**3,
             alpha = 0.3,
             color = cmc.cm.romaO(0.80)
            )
    plt.hist(von_neumann_filtered3_df['equivalent_diameter'], 
             bins = BINS_FOR_HISTOGRAM, 
             density = True, 
             weights = ((4/3) * np.pi) * von_neumann_filtered3_df['equivalent_diameter']**3,
             alpha = 0.3,
             color = cmc.cm.romaO(0.15)
            )
    
    plt.xlabel('vol-wt\'d Equivalent Diameter (microns)', fontsize = ax_font_size, fontweight = 'bold')
    plt.ylabel('Density',  fontsize = ax_font_size, fontweight = 'bold')
    plt.xticks(fontsize=ax_font_size, fontweight='bold')
    plt.yticks(fontsize=ax_font_size, fontweight='bold')
    plt.box(True)
    plt.gca().spines['top'].set_linewidth(4)    # Set top spine thickness
    plt.gca().spines['bottom'].set_linewidth(4) # Set bottom spine thickness
    plt.gca().spines['left'].set_linewidth(4)   # Set left spine thickness
    plt.gca().spines['right'].set_linewidth(4)  # Set right spine thickness
    plt.tick_params(axis='both', direction='in', length=10)  # Set the length of tick marks
    
    ax_vol_psd.set_ylim([0, 1])
    ax_vol_psd.set_xlim([0, 10])
    
    vol_legend = plt.legend(labels = ['voroni_otsu', 'von neumann'], framealpha=1, frameon=True, fontsize=ax_font_size - 4)
    frame = vol_legend.get_frame()
    frame.set_edgecolor('black')  # Set legend box color
    frame.set_linewidth(2)        # Set legend box thickness
    for text in vol_legend.get_texts():
        text.set_fontweight('bold')

    # Add a column for volume (proportional to the cube of the diameter)
    von_neumann_filtered3_df['Volume'] = ((4/3) * np.pi) * von_neumann_filtered3_df['equivalent_diameter'] ** 3 # CORRECT THIS TO BE THE TRUE VOLUME EQUATION
    
    # Calculate cumulative volume fraction
    von_neumann_filtered3_df = von_neumann_filtered3_df.sort_values(by='equivalent_diameter').reset_index(drop=True)
    von_neumann_filtered3_df['Volume_Cumulative'] = von_neumann_filtered3_df['Volume'].cumsum() / von_neumann_filtered3_df['Volume'].sum()
    
    # "Interpolating volume-weighted d10, d50, and d90", the numbers its spitting out are probably not interpolated like the they should be but its close enough if the dataset is large enough
    d10_vol_von_neumann = von_neumann_filtered3_df.loc[von_neumann_filtered3_df['Volume_Cumulative'] >= 0.10, 'equivalent_diameter'].iloc[0]
    d50_vol_von_neumann = von_neumann_filtered3_df.loc[von_neumann_filtered3_df['Volume_Cumulative'] >= 0.50, 'equivalent_diameter'].iloc[0]
    d90_vol_von_neumann = von_neumann_filtered3_df.loc[von_neumann_filtered3_df['Volume_Cumulative'] >= 0.90, 'equivalent_diameter'].iloc[0]
    span_vol_von_neumann = (d90_vol_von_neumann - d10_vol_von_neumann) / d50_vol_von_neumann
    mean_vol_von_neumann = np.sum(von_neumann_filtered3_df['equivalent_diameter'] * von_neumann_filtered3_df['Volume']) / von_neumann_filtered3_df['Volume'].sum() # This is the same as the D[5,3], volume weighted size
    std_vol_von_neumann = np.sqrt(np.sum((von_neumann_filtered3_df['equivalent_diameter'] - mean_vol_von_neumann)**2 * von_neumann_filtered3_df['Volume']) / von_neumann_filtered3_df['Volume'].sum()) #This is the calculation suggest for weighted standard deviation based on wikipedia and a stackoverflow discussion
    
    print("\nvon Neumann, Volume-weighted distribution:")
    print(f"d10, von Neumann (volume-weighted): {d10_vol_von_neumann}")
    print(f"d50, von Neumann (volume-weighted): {d50_vol_von_neumann}")
    print(f"d90, von Neumann (volume-weighted): {d90_vol_von_neumann}")
    print(f"Span, von Neumann (volume-weighted): {span_vol_von_neumann}")
    print(f"Mean, von Neumann (volume-weighted): {mean_vol_von_neumann}")
    print(f"Std Deviation, von Neumann (volume-weighted): {std_vol_von_neumann}")

    
    # Add a column for volume (proportional to the cube of the diameter)
    voroni_otsu_filtered3_df['Volume'] = ((4/3) * np.pi) * voroni_otsu_filtered3_df['equivalent_diameter'] ** 3 # CORRECT THIS TO BE THE TRUE VOLUME EQUATION
    
    # Calculate cumulative volume fraction
    voroni_otsu_filtered3_df = voroni_otsu_filtered3_df.sort_values(by='equivalent_diameter').reset_index(drop=True)
    voroni_otsu_filtered3_df['Volume_Cumulative'] = voroni_otsu_filtered3_df['Volume'].cumsum() / voroni_otsu_filtered3_df['Volume'].sum()
    
    # "Interpolating volume-weighted d10, d50, and d90", the numbers its spitting out are probably not interpolated like the they should be but its close enough if the dataset is large enough
    d10_vol_voroni_otsu = voroni_otsu_filtered3_df.loc[voroni_otsu_filtered3_df['Volume_Cumulative'] >= 0.10, 'equivalent_diameter'].iloc[0]
    d50_vol_voroni_otsu = voroni_otsu_filtered3_df.loc[voroni_otsu_filtered3_df['Volume_Cumulative'] >= 0.50, 'equivalent_diameter'].iloc[0]
    d90_vol_voroni_otsu = voroni_otsu_filtered3_df.loc[voroni_otsu_filtered3_df['Volume_Cumulative'] >= 0.90, 'equivalent_diameter'].iloc[0]
    span_vol_voroni_otsu = (d90_vol_voroni_otsu - d10_vol_voroni_otsu) / d50_vol_voroni_otsu
    mean_vol_voroni_otsu = np.sum(voroni_otsu_filtered3_df['equivalent_diameter'] * voroni_otsu_filtered3_df['Volume']) / voroni_otsu_filtered3_df['Volume'].sum() # This is the same as the D[5,3], volume weighted size
    std_vol_voroni_otsu = np.sqrt(np.sum((voroni_otsu_filtered3_df['equivalent_diameter'] - mean_vol_voroni_otsu)**2 * voroni_otsu_filtered3_df['Volume']) / voroni_otsu_filtered3_df['Volume'].sum()) #This is the calculation suggest for weighted standard deviation based on wikipedia and a stackoverflow discussion
    
    print("\nVoroni-Otsu, Volume-weighted distribution:")
    print(f"d10, Voroni-Otsu (volume-weighted): {d10_vol_voroni_otsu}")
    print(f"d50, Voroni-Otsu (volume-weighted): {d50_vol_voroni_otsu}")
    print(f"d90, Voroni-Otsu (volume-weighted): {d90_vol_voroni_otsu}")
    print(f"Span, Voroni-Otsu (volume-weighted): {span_vol_voroni_otsu}")
    print(f"Mean, Voroni-Otsu (volume-weighted): {mean_vol_voroni_otsu}")
    print(f"Std Deviation, Voroni-Otsu (volume-weighted): {std_vol_voroni_otsu}")
    
    
def save_domain_size_df(domain_size_df, df_type, experiment_id):
    """ 
    Offers an option to save the statistical data as a CSV for compilation of data later 
    """ 
    domain_size_df.to_csv(f"{experiment_id}_{df_type}.csv")


print("DONE")

In [ ]:
#Don't worry about it, just hit Shift+Enter and grab the image you want to analyze
image, image_micron_scale = import_then_process_img()

## Okay, now we have an image to work with for the next steps...

The next step will spit out a series of thresholds, select the one which includes the active material and goes a bit over with including extra material -- we will wash out that extra material.

In [ ]:
# Don't worry about it, just hit Shift+Enter 
img_binary = try_all_thresholds(image)

## Now comes image manipulation to remove the extra

We are looking to remove enough of the extra to have distinct domains that are not connected by what is clearly junk (likely binder/SE/carbon) -- this does not have to be perfect as we have ways of removing that noise later

In [ ]:
# Don't worry about it, just hit Shift+Enter, or change the erosion range value if you're not happy with the options
# if erosion is not necessary then have the value equal 0
treated_img = img_erosion_n_fill(img_binary, erosion_range = 4)

In [ ]:
##THESE FUNCTIONS SHOULD BE COMBINED TOGETHER INTO ONE

binary_labels = img_binary_labeling(treated_img,
                                    spot_sigma_val = 15,
                                    outline_sigma_val = 0)

list_dfs = img_binary_enframing(image, 
                                von_neumann_lbls = binary_labels[0], 
                                voroni_otsu_lbls = binary_labels[1],
                                micron_per_pixel = image_micron_scale)

## Time to remove the remaining noise we picked up during labeling

Below will be ECDF plots -- use your best judgement as to what is reasonable cutt off for equivalent diameter

https://en.wikipedia.org/wiki/Empirical_distribution_function

In [ ]:
# Don't worry about it, just hit Shift+Enter
cutoff_particle_size(von_neumann_df = list_dfs[0], 
                     voroni_otsu_df = list_dfs[1]) #CHANGE TO HAVE A RETURN VALUE FOR THE CUTOFF

In [ ]:
# Change the cutoff_value to a size that you think will remove the noise while retaining the particles
# for cathode that is probably around 0.5 to 0.7

list_dfs_filtered = cutoff_particle_apply(von_neumann_df = list_dfs[0],
                                          voroni_otsu_df = list_dfs[1],
                                          cutoff_type = "size",
                                          cutoff_value = 0.7)

cutoff_particles_on_edge(von_neumann_filtered_df = list_dfs_filtered[0], 
                         voroni_otsu_filtered_df = list_dfs_filtered[1]) #CHANGE TO HAVE A RETURN VALUE FOR THE CUTOFF

In [ ]:
# Change the cutoff_value to a decimal that will remove the particles whose true size is obfuscated by being on the edge
# my opinion is that anything with 0.15 or more should be discounted

list_dfs_filtered2 = cutoff_particle_apply(von_neumann_df = list_dfs_filtered[0], 
                                           voroni_otsu_df = list_dfs_filtered[1], 
                                           cutoff_type = "edge", 
                                           cutoff_value = 0.15)

list_dfs_filtered3 = finalize_data(von_neumann_filtered2_df = list_dfs_filtered2[0], 
                                   voroni_otsu_filtered2_df = list_dfs_filtered2[1])

In [ ]:
#Add notes for how to work this
plot_and_process_particle_data(image, 
                               von_neumann_filtered3_df = list_dfs_filtered3[0], 
                               von_neumann_lbls = binary_labels[0], 
                               voroni_otsu_filtered3_df = list_dfs_filtered3[1], 
                               voroni_otsu_lbls = binary_labels[1],
                               experiment_id_for_file = "test",
                               consolidated_data = False
                              )